In [4]:
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.optimizers import SGD, Adam
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from sklearn.manifold import TSNE
from IPython.display import clear_output
from keras.regularizers import l2

import os
import numpy as np

Using TensorFlow backend.


In [1]:
labels = {'agricultural':0, 'airplane':1, 'baseballdiamond':3, 'beach':4, 'buildings':5, 'chaparral':6, 
          'denseresidential':7, 'forest':8, 'freeway':9, 'golfcourse':10, 'harbor':11, 'intersection':12,
         'mediumresidential':13, 'mobilehomepark':15, 'overpass':16, 'parkinglot':17, 'river':18,
         'runway':18, 'sparseresidential':19, 'storagetanks':20, 'tenniscourt':21, }

def load_images(dataset_path, total_images, labels, input_shape):
    print(input_shape)
    w = input_shape[0]
    h = input_shape[1]
    print(w,h)
    shape = (total_images+1,input_shape[0],input_shape[1],input_shape[2])
    y_shape = (total_images+1,1)
    x = np.zeros(shape)
    y = np.zeros(y_shape)
    folders = os.listdir(dataset_path)
    i = 0
    h = 0
    for folder in folders:
        folder_path = os.path.join(dataset_path, folder)
        images = os.listdir(folder_path)
        h+=1
        for image in images:
            # Lembrando que todas as imagens possuem o formato labelxx.jpg
#             print(image)
#             print(folder)
            image_path = os.path.join(folder_path,image)
            img_np = cv2.imread(image_path)
            img_np = np.resize(img_np, input_shape)
            #print(img_np.shape)
            x[i] = img_np
            y[i] = np.array( int(labels[folder]) )
            i+=1
            del img_np
    return x, y

In [2]:
def Train(net, x_train, y_train_h, x_test, y_test_h, batch_size, epochs): 
  
    #Computing the number of batchs
    n_batchs_tr = x_train.shape[0]//batch_size
    n_batchs_ts = x_test.shape[0]//batch_size
    
    acc_train = []
    loss_train = []
    acc_test = []
    loss_test = []
    
    print('Start the training')
    for epoch in range(epochs):
        loss_tr = np.zeros((1 , 2))
        loss_ts = np.zeros((1 , 2))
        #Random shuffle the train data 
        x_train , y_train_h = shuffle(x_train , y_train_h , random_state = 0)

        # Training the network per batch
        for  batch in range(n_batchs_tr):

            x_train_b = x_train[batch * batch_size : (batch + 1) * batch_size , : , : , :]
            y_train_h_b = y_train_h[batch * batch_size : (batch + 1) * batch_size , : ]

            loss_tr = loss_tr + net.train_on_batch(x_train_b , y_train_h_b)

        acc_train.append(100*(loss_tr[0,1]/n_batchs_tr))
        loss_train.append(loss_tr[0,0]/n_batchs_tr)

        loss_tr = loss_tr/n_batchs_tr

        # Evaluating the network in the test set
        for  batch in range(n_batchs_ts):

            x_test_b = x_test[batch * batch_size : (batch + 1) * batch_size , : , : , :]
            y_test_h_b = y_test_h[batch * batch_size : (batch + 1) * batch_size , : ]

            loss_ts = loss_ts + net.test_on_batch(x_test_b , y_test_h_b)

        acc_test.append(100*(loss_ts[0,1]/n_batchs_ts))
        loss_test.append(loss_ts[0,0]/n_batchs_ts)

        loss_ts = loss_ts/n_batchs_ts

        print("%d [Train loss: %f , Train acc.: %.2f%%][Test loss: %f , Test acc.:%.2f%%]"
             %(epoch , loss_tr[0 , 0], 100*loss_tr[0 , 1] , loss_ts[0 , 0] , 100 * loss_ts[0 , 1]))  

        #Visualize Results every 5 epochs
        if epoch>1 and epoch%5==0:
            graph_training_history(acc_train, acc_test, loss_train, loss_test)

    return net

In [3]:
if __name__=='__main__':
  
    # Define shapes
    input_shape = (64 , 64 , 3)
    classes = 21
    
    #Loading the data set
    train_path = "/home/thimabru/Deep_Learning_aula/aula4/cyberlab_mission/UCMERCED_HW_SceneClassification/data/separated_in_labels/train"
    total_train_images = 1332 - 1
    x_train, y_train = load_images(train_path, total_train_images, labels, input_shape)

    test_path = "/home/thimabru/Deep_Learning_aula/aula4/cyberlab_mission/UCMERCED_HW_SceneClassification/data/separated_in_labels/val"
    total_test_images = 363 - 1
    x_test, y_test = load_images(test_path, total_train_images, labels, input_shape)
    
    print(y_train.shape)

    #Normalizing the set
    x_train = x_train.astype('float32')/255.
    x_test = x_test.astype('float32')/255.
    
    y_train = y_train.astype('uint8')
    y_test = y_test.astype('uint8')

    #Performing hot encoding
    y_train_h = keras.utils.to_categorical(y_train , classes)
    y_test_h = keras.utils.to_categorical(y_test , classes)
    


    ############ Change here  #########
    lr = 1e-4
    batch_size = 64
    epochs = 30
    adam = Adam(lr = lr)

    # Create network
    net = CNN(input_shape, classes)
    net.compile(loss = 'categorical_crossentropy', optimizer=adam , metrics=['accuracy'])
    net.summary()

    #Call the train function  
    net = Train(net, x_train, y_train_h, x_test, y_test_h, batch_size, epochs)

(64, 64, 3)
64 64


NameError: name 'np' is not defined